In [2]:
import os
import torch
from torch import nn
from PIL import Image
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

from clipseg_model import CLIPSeg
from data.cub_parts_dataset import CUBPartsDataset, collate_fn

%load_ext autoreload
%autoreload 2

clip_seg = CLIPSeg()
state_dict = torch.load('checkpoints/clipseg_pascub_ft.pt', map_location='cpu')
clip_seg.load_state_dict(state_dict)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Some weights of CLIPSegForImageSegmentation were not initialized from the model checkpoint at CIDAS/clipseg-rd64-refined and are newly initialized: ['non_object_embedding', 'text_adapter.fc.0.weight', 'text_adapter.fc.2.weight', 'tunable_linear.weight', 'visual_adapter.fc.0.weight', 'visual_adapter.fc.2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<class 'clipseg.configuration_clipseg.CLIPSegConfig'> 512


<All keys matched successfully>

In [5]:
dataset_test = CUBPartsDataset('datasets/PASCUB', split='test')
dataloader_test = DataLoader(dataset=dataset_test, batch_size=2, shuffle=False, collate_fn=collate_fn)
dataloader_test_iter = iter(dataloader_test)

In [13]:
# inference
batch = next(dataloader_test_iter)
imag_paths, gt_paths, imgs, gts = batch
clip_seg.eval()
with torch.no_grad():
    logits, outputs = clip_seg(imgs)

activation.shape torch.Size([16, 485, 768])
activation.shape torch.Size([16, 485, 768])
torch.Size([16, 485, 64]) torch.Size([16, 485, 64])


TypeError: cannot unpack non-iterable NoneType object

In [27]:
a = torch.stack([torch.full((768,), i) for i in range(20)])
print(a.shape)
a = a.unsqueeze(0).repeat(4, 1, 1).reshape(-1, 768)
print(a.shape)
a[1, ...]

torch.Size([20, 768])
torch.Size([80, 768])


tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
from clipseg.modeling_clipseg import CLIPSegDecoderLayer, CLIPSegDecoder


decoder_layer = nn.TransformerDecoderLayer(d_model=64, nhead=4, dim_feedforward=2048, activation='relu', norm_first=False)
# decoder = nn.TransformerDecoder(decoder_layer=..., num_layers=3, norm=...)

In [ ]:
x = torch.randn(32, 485, 64) # [bs*num_cond_emb, num_tokens, reduce_dim]
y = torch.randn(128, 485, 64) # [bs*num_cond_emb, num_tokens, reduce_dim]
decoder_layer(x, y).shape

In [ ]:
from clipseg.configuration_clipseg import CLIPSegConfig

config = CLIPSegConfig()

In [ ]:
config.vision_config.hidden_size, config.vision_config.intermediate_size

In [ ]:
clip_seg = CLIPSeg()

In [ ]:
a = torch.tensor([[i] * 10 for i in range(10)])
a

In [5]:
import matplotlib.pyplot as plt

In [6]:
image.shape

torch.Size([3, 333, 500])